<H1>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MACHINE LEARNING PROJESİ</H1>

## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;NLP (Natural Language Processing) KULLANARAK IMDB FİLM 

## YORUMLARI KAGGLE DATA SETİ ÜZERİNDE SENTIMENT(DUYGU) ANALİZİ


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords



In [2]:
# Veri setlerimizi yüklüyoruz..
df = pd.read_csv('NLPlabeledData.tsv',  delimiter="\t", quoting=3)



In [3]:
# Verimize bakalım
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
len(df)

25000

In [5]:
len(df["review"])

25000

In [6]:
# stopwords'ü temizlemek için nltk kütüphanesinden stopwords kelime setini bilgisayarımıza indirmemiz gerekiyor. 
# Bu işlemi nltk ile yapıyoruz
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sungu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## * * * * Veri Temizleme İşlemleri * * * *

### Öncelikle BeautifulSoup modülünü kullanarak HTML taglerini review cümlelerinden sileceğiz.
Bu işlemlerin nasıl yapıldığını açıklamak için önce örnek tek bir review seçip size nasıl yapıldığına bakalım:

In [ ]:
sample_review= df.review[0]
sample_review

In [ ]:
# HTML tagleri temizlendikten sonra..
sample_review = BeautifulSoup(sample_review).get_text()
sample_review

In [ ]:
# noktalama işaretleri ve sayılardan temizliyoruz - regex kullanarak..
sample_review = re.sub("[^a-zA-Z]",' ',sample_review)
sample_review

In [ ]:
# küçük harfe dönüştürüyoruz, makine öğrenim algoritmalarımızın büyük harfle başlayan kelimeleri farklı kelime olarak
# algılamaması için yapıyoruz bunu:
sample_review = sample_review.lower()
sample_review

In [ ]:
# stopwords (yani the, is, are gibi kelimeler yapay zeka tarafından kullanılmamasını istiyoruz. Bunlar gramer kelimeri..)
# önce split ile kelimeleri bölüyoruz ve listeye dönüştürüyoruz. amacımız stopwords kelimelerini çıkarmak..
sample_review = sample_review.split()


  

In [ ]:
sample_review

In [ ]:
len(sample_review)

In [ ]:
# sample_review without stopwords
swords = set(stopwords.words("english"))                      # conversion into set for fast searching
sample_review = [w for w in sample_review if w not in swords]               
sample_review


In [ ]:
len(sample_review)

In [ ]:
# Temizleme işlemini açıkladıktan sonra şimdi tüm dataframe'imiz içinde bulunan reviewleri döngü içinde topluca temizliyoruz
# bu amaçla önce bir fonksiyon oluşturuyoruz:

In [7]:
def process(review):
    # review without HTML tags
    review = BeautifulSoup(review).get_text()
    # review without punctuation and numbers
    review = re.sub("[^a-zA-Z]",' ',review)
    # converting into lowercase and splitting to eliminate stopwords
    review = review.lower()
    review = review.split()
    # review without stopwords
    swords = set(stopwords.words("english"))                      # conversion into set for fast searching
    review = [w for w in review if w not in swords]               
    # splitted paragraph'ları space ile birleştiriyoruz return
    return(" ".join(review))

In [8]:
# training datamızı yukardaki fonksiyon yardımıyla temizliyoruz: 
# her 1000 review sonrası bir satır yazdırarak review işleminin durumunu görüyoruz..

train_x_tum = []
for r in range(len(df["review"])):        
    if (r+1)%1000 == 0:        
        print("No of reviews processed =", r+1)
    train_x_tum.append(process(df["review"][r]))

No of reviews processed = 1000
No of reviews processed = 2000
No of reviews processed = 3000
No of reviews processed = 4000
No of reviews processed = 5000
No of reviews processed = 6000
No of reviews processed = 7000
No of reviews processed = 8000
No of reviews processed = 9000
No of reviews processed = 10000
No of reviews processed = 11000
No of reviews processed = 12000
No of reviews processed = 13000
No of reviews processed = 14000
No of reviews processed = 15000
No of reviews processed = 16000
No of reviews processed = 17000
No of reviews processed = 18000
No of reviews processed = 19000
No of reviews processed = 20000
No of reviews processed = 21000
No of reviews processed = 22000
No of reviews processed = 23000
No of reviews processed = 24000
No of reviews processed = 25000


### Train, test split...

In [9]:
x = train_x_tum
y = np.array(df["sentiment"])

# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1)


### Bag of Words oluşturuyoruz !

Verilerimizi temizledik ancak yapay zekanın çalışması için bu metin tabanlı verileri sayılara ve bag of words denilen matrise çevirmek gerekiyor. İşte bu amaçla sklearn içinde bulunan CountVectorizer aracını kullanıyoruz:

<IMG src="bag.jpg" width="900" height="900" >

In [10]:
# sklearn içinde bulunan countvectorizer fonksiyonunu kullanarak max 5000 kelimelik bag of words oluşturuyoruz...
vectorizer = CountVectorizer( max_features = 5000 )

# train verilerimizi feature vektöre matrisine çeviriyoruz
train_x = vectorizer.fit_transform(train_x)



In [11]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1775691 stored elements in Compressed Sparse Row format>

In [12]:
# Bunu array'e dönüştürüyoruz çünkü fit işlemi için array istiyor..
train_x = train_x.toarray()
train_y = y_train

In [13]:
train_x.shape, train_y.shape

((22500, 5000), (22500,))

In [14]:
train_y

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

### Random Forest Modeli oluşturuyoruz ve fit ediyoruz

In [15]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)
model.fit(train_x, train_y)


RandomForestClassifier(random_state=42)

### Şimdi sıra test datamızda..

In [16]:
# Test verilerimizi feature vektöre matrisine çeviriyoruz
# Yani aynı işlemleri(bag of wordse dönüştürme) tekrarlıyoruz bu sefer test datamız için:
test_xx = vectorizer.transform(test_x)


In [17]:
test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 199248 stored elements in Compressed Sparse Row format>

In [18]:
test_xx = test_xx.toarray()

In [19]:
test_xx.shape

(2500, 5000)

#### Prediction yapıyoruz..

In [20]:
test_predict = model.predict(test_xx)
dogruluk = roc_auc_score(y_test, test_predict)

In [21]:
print("Doğruluk oranı : % ", dogruluk * 100)

Doğruluk oranı : %  83.82325927319812
